![Clothing Classifier Model](Clothing%20Classifier%20Model.png)


Fashion Forward is a new AI-based e-commerce clothing retailer.
They want to use image classification to automatically categorize new product listings, making it easier for customers to find what they're looking for. It will also assist in inventory management by quickly sorting items.

As a data scientist tasked with implementing a garment classifier, your primary objective is to develop a machine learning model capable of accurately categorizing images of clothing items into distinct garment types such as shirts, trousers, shoes, etc.


In [80]:
# Run the cells below first

In [81]:
!pip install torchmetrics

Defaulting to user installation because normal site-packages is not writeable


In [82]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy, Precision, Recall, ConfusionMatrix

In [83]:
# Load datasets
from torchvision import datasets
import torchvision.transforms as transforms

train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

In [84]:
# Convert datasets to DataLoader
dataloader_train = DataLoader(
  train_data, shuffle=True, batch_size=10
)

dataloader_test = DataLoader(
  test_data, shuffle=False, batch_size=1
)


In [85]:
# Define Global Variables
# Get the number of classes
classes = train_data.classes
num_classes = len(classes)

# Define some relevant variables
num_input_channels = 1
num_output_channels = 16
image_size = train_data[0][0].shape[1]

# Print key inputs
print(f"The number of classes are :{num_classes}")
print(f"The classes are : {classes}")
print(f"The Number of Input Channels are {num_input_channels}, the Number of Output Channel are {num_output_channels}, and the Image Size is {image_size}")
print(f"The length of train is {len(dataloader_train)} and the lenth of test is {len(dataloader_test)}")


The number of classes are :10
The classes are : ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
The Number of Input Channels are 1, the Number of Output Channel are 16, and the Image Size is 28
The length of train is 6000 and the lenth of test is 10000


# Defining the CNN (Convolutional Neural Network)

In [86]:
class Net(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        # Define feature extractor
        self.feature_extractor = nn.Sequential(
        # Convolution layer with kernel size 3 and stride 1
        nn.Conv2d(in_channels=num_input_channels, out_channels=num_output_channels, kernel_size=3, stride=1, padding=1),
        # RectiLinear unit activation layer
        nn.ReLU(),
        # Max pooling layer with kernel size 2 and stride 2
        nn.MaxPool2d(kernel_size=2, stride=2),
        # Flatten
        nn.Flatten(),
        )
        # Define classifier
        # Fully connected layer
        self.classifier = nn.Linear(num_output_channels*(image_size//2)**2, num_classes)
    
    def forward(self, x):  
        # Pass input through feature extractor and classifier
        x = self.feature_extractor(x)
        x = self.classifier(x)
        return x

# Train the CNN

In [87]:
# Define the model
net = Net(num_classes=num_classes)
# Define the loss function
criterion = nn.CrossEntropyLoss()
# Define the optimizer
optimizer = optim.Adam(net.parameters(), lr=0.001)

for epoch in range(1):
    running_loss = 0.0
    # Iterate over training batches
    for images, labels in dataloader_train:
        optimizer.zero_grad()
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    epoch_loss = running_loss / len(dataloader_train)
    print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}")

Epoch 1, Loss: 0.3965


# Testing the CNN

In [88]:
# Define metrics
accuracy_metric = Accuracy(task='multiclass', num_classes=num_classes)
precision_metric = Precision(task='multiclass', num_classes=num_classes, average=None)
recall_metric = Recall(task='multiclass', num_classes=num_classes, average=None)
metric_confusion_matrix = ConfusionMatrix(task="multiclass", num_classes=num_classes)


# Run model on test set
net.eval()
predicted = []
for i, (features, labels) in enumerate(dataloader_test):
    output = net.forward(features.reshape(-1, 1, image_size, image_size))
    cat = torch.argmax(output, dim=-1)
    predicted.extend(cat.tolist())
    accuracy_metric(cat, labels)
    precision_metric(cat, labels)
    recall_metric(cat, labels)

# Compute the metrics
accuracy = accuracy_metric.compute().item()
precision = precision_metric.compute().tolist()
recall = recall_metric.compute().tolist()
print('Accuracy:', accuracy)
print('Precision (per class):', precision)
print('Recall (per class):', recall)
print(confusion_matrix)

Accuracy: 0.8830999732017517
Precision (per class): [0.7856502532958984, 0.9917440414428711, 0.8417112231254578, 0.8743914365768433, 0.80525803565979, 0.9747983813285828, 0.6832432150840759, 0.9419291615486145, 0.9709709882736206, 0.9608040452003479]
Recall (per class): [0.8759999871253967, 0.9610000252723694, 0.7870000004768372, 0.8980000019073486, 0.8270000219345093, 0.9670000076293945, 0.6320000290870667, 0.9570000171661377, 0.9700000286102295, 0.9559999704360962]
tensor([[881,   1,  17,  20,   5,   1,  61,   0,  14,   0],
        [  2, 974,   1,  16,   4,   0,   2,   0,   1,   0],
        [ 19,   1, 832,   6,  78,   0,  60,   0,   4,   0],
        [ 30,  16,  15, 879,  27,   0,  30,   0,   3,   0],
        [  1,   1,  80,  32, 819,   0,  66,   0,   1,   0],
        [  0,   0,   0,   1,   0, 959,   0,  18,   2,  20],
        [162,   2,  89,  27,  74,   0, 625,   0,  21,   0],
        [  0,   0,   0,   0,   0,  13,   0, 891,   1,  95],
        [  2,   1,   1,   3,   2,   2,   9,   2,